&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;
    
    
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;


# Introduction to DeepStream

In this notebook, you will be introduced to DeepStream ,it's workflow and the underlying principle upon which it works on. 


**Contents of this Notebook :**

- [DeepStream](#DeepStream) 
- [Overview of the DeepStream SDK](#Overview-of-the-DeepStream-SDK)
- [GStreamer Foundations](#GStreamer-Foundations-:)
    - [Elements](#Elements)
    - [Pipeline](#Pipeline)
    - [Pads](#Pads)
    - [Caps](#Caps)
    - [Buffers](#Buffers)
    - [Plugin based Architecture](#Plugin-based-Architecture)
    

## DeepStream 

![Worlflow](images/ds-workflow.jpg)

DeepStream simplifies building IVA applications by separating the application into components managed and built by the user and components managed by DeepStream.

As developers, we build components to manage important business tasks like :
- Selecting the kind and number of video streams we want to analyze
- Choosing the type of analysis, we want to do on the video
- Handling and interacting with the results of our analysis
    
We don't need to build components to manage difficult tasks like :
- Efficiently leverage the GPU for accelerated processing and inference
- Efficiently process data from multiple video streams at once
- Keeping track of metadata associated with each frame of video from multiple sources
- Optimizing our pipeline for maximum data throughput
- Optimizing our neural networks for high-speed inference

These are mundane tasks that the **DeepStream SDK** manages for us. That lets us focus on the more important tasks related to the project's goal and impact. DeepStream lets us focus on the intelligence portion of the application.

Here is an illustration on DeepStream Workflow that describes the tasks by DeepStream vs developer 

![Workflow_split](images/ds-workflow-split.jpeg)


### Performance and Scalability 


### Performance  

To give an overview of performance improvement when using DeepStream SDK, we will scrutinize the DeepStream-app reference application included within the release package. The below illustration shows that the performance is doubled using DeepStream 3.0 when tested on T4 compared to the previous version P4, while it consumes the same amount of power and equal number of streams.The reference application includes a primary detector, three classifiers and a tracker.

![Performance](images/ds-perf.png)


### Scalability 

DeepStream provides scalability at different levels of the system hierarchy. 

For example, 
- The DeepStream SDK 3.0 supports processing a higher number of concurrent streams, in addition to utilizing multiple GPUs upon availability. 
- The DeepStream SDK 4.0 delivers a unified code base for all NVIDIA GPUs and quick integration with IoT services.

Furthermore DeepStream-in-containers provide flexibility to the deployment phase as shown below:

![Scalability](images/ds-scalability.png)

# Overview of the DeepStream SDK

The DeepStream SDK consists of a set of building blocks which bridge the gap between low level APIs (such as TensorRT, Video Codec SDK) and the user application. By utilizing the DeepStream SDK, you can accelerate development of the IVA applications by focusing on building core deep learning models instead of designing end-to-end applications from scratch.

Below, you can see a schematic presentation of the DeepStream SDK in a series of potential
applications.

![SDK](images/ds-sdk.png)


In addition, the DeepStream SDK extends these capabilities by providing several other hardware accelerated building blocks. This includes support for TensorRT 7 and CUDA 11. In addition, DeepStream applications can be deployed as a part of a larger multi-GPU cluster or a microservice in containers. This allows highly flexible system architectures and opens new application capabilities.

Below, you can see a shortened list of new capabilities provided by DeepStream:
- Allowing addition and removal of video stream dynamically and during the pipeline execution,in addition to frame rate and resolution adjustments
- Extending the video processing capabilities by supporting custom layers, and user-defined parsing of detector outputs
- Providing Support for 360-degree camera using GPU-accelerated dewarping libraries
- Augmenting the meta-data with application-specific, user-defined insights
- Providing pruned and efficient inference models
- Getting detailed performance analysis with the NVIDIA Nsight system profiler tool.


The DeepStream SDK is based on the **GStreamer multimedia framework** and provides a pipeline of GPU accelerated plugins as shown below. The SDK facilitates application implementation procedure by providing plugins for video inputs, video decoding, image preprocessing, TensorRT-based inference, object tracking and display. You can utilize these capabilities to assemble flexible, multi-stream video analytics applications.

![Sample_pipeline](images/ds-sample-pipeline.png)

# GStreamer Foundations :

The DeepStream SDK is based on the open source [GStreamer multimedia framework](https://gstreamer.freedesktop.org/). There are a few key concepts in GStreamer that we need to
touch on before getting started. These include Pipelines,Elements, Pads, Buffers, and Caps. We will be
describing them at a high level, but encourage those who are interested in the details to read the
[GStreamer Basics](https://gstreamer.freedesktop.org/documentation/?gi-language=c) documentation to learn more.


### Elements 

Elements are the core building block with which we make pipelines. Every process in-between the source (i.e. input of the pipeline, e.g. camera and video files) and sink elements (e.g. screen display) is passed through elements. Video decoding and encoding, neural network inference, and displaying text on top of video streams are examples of "element". DeepStream allows us to instantiate elements and weave them into pipelines.


### Pipeline 

All elements in GStreamer must typically be contained inside a pipeline before they can be used, because it takes care of some clocking and messaging functions.

A pipeline is a particular type of bin, which is the element used to contain other elements. Therefore all methods which apply to bins also apply to pipelines. We need to add the elements to the pipeline and are then linked ,this linking must be established following the data flow (this is, from source elements to sink elements). 

![pipeline](images/pipeline.png)

### Pads

Pads are the interfaces between elements. When data flows from element to another element in a pipeline, it flows from the sink pad of one element to the source pad of another. Note that each element might have zero, one or many source/sink elements.


![pads](images/pads.png)

### Caps

Caps (or Capabilities), are the data types that a pad is permitted to utilize or emit. Because pads
can allow multiple data types, sometimes the data flow is ambiguous. Pads are "negotiated" in
order to explicitly define the type of data that can flow through the pad. Caps streamline this
process and allow elements of our pipeline with ambiguous pads to negotiate the correct data flow
process. Later in this course, we will use caps to pass certain video data types (NV12, RGB) to the
downstream elements in the pipeline.

### Buffers

Buffers carry the data that will passed on through the pipeline. Buffers are timestamped, contain
metadata such as how many elements are using it, flags, and pointers to objects in memory. When
we write application code, we rely on accessing data attached to the buffer.

### Plugin based Architecture

DeepStream applications can be thought of as pipelines consisting of individual components
(plugins). Each plugin represents a functional block like inference using TensorRT or multi-stream
decode. Where applicable, plugins are accelerated using the underlying hardware to deliver
maximum performance. DeepStream’s key value is in making deep learning for video easily
accessible, to allow you to concentrate on quickly building and customizing efficient and scalable
video analytics applications.

The plugin architecture provides functionality such as video encode/decode, scaling, inferencing, and more. By connecting plugins into a pipeline, we can build complex applications. Because DeepStream is built on top of GStreamer, we can inspect plugins using `gst-inspect-1.0`.


In [1]:
# Inspect the nvinfer plugin
!gst-inspect-1.0 nvinfer

Factory Details:
  Rank                     primary (256)
  Long-name                NvInfer plugin
  Klass                    NvInfer Plugin
  Description              Nvidia DeepStreamSDK TensorRT plugin
  Author                   NVIDIA Corporation. Deepstream for Tesla forum: https://devtalk.nvidia.com/default/board/209

Plugin Details:
  Name                     nvdsgst_infer
  Description              NVIDIA DeepStreamSDK TensorRT plugin
  Filename                 /usr/lib/x86_64-linux-gnu/gstreamer-1.0/deepstream/libnvdsgst_infer.so
  Version                  6.0.0
  License                  Proprietary
  Source module            nvinfer
  Binary package           NVIDIA DeepStreamSDK TensorRT plugin
  Origin URL               http://nvidia.com/

GObject
 +----GInitiallyUnowned
       +----GstObject
             +----GstElement
                   +----GstBaseTransform
                         +----GstNvInfer

Pad Templates:
  SRC template: 'src'
    Availability: Always
    Capa

 ## Licensing
  
This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).
    
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;
